In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [3]:
data_item = pd.read_pickle('/kaggle/input/neu-mf/data_item.pkl')
data_rec = pd.read_pickle('/kaggle/input/neu-mf/data_rec.pkl')
data_user = pd.read_pickle('/kaggle/input/neu-mf/data_user.pkl')

In [3]:
data_user['idx_user'] = [i for i in range(len(data_user))]
data_item['idx_item'] = [i for i in range(len(data_item))]

In [4]:
data = pd.merge(data_rec,data_user[['idx_user','user_id']],on ='user_id')
data = pd.merge(data,data_item[['idx_item','app_id']], on = 'app_id')

In [5]:
del data['app_id']
del data['user_id']

In [6]:
data.head()

,is_recommended,hours,idx_user,idx_item
0,1,0.0097,400,4087
1,1,0.0102,56321,4087
2,1,0.0083,8689,4087
3,1,0.0079,22779,4087
4,1,0.0169,8787,4087


* train test split

In [10]:
# train with percent
data_train = pd.DataFrame()
data_test = pd.DataFrame() 
for key, val in data.groupby('idx_item'): 
  a = int(len(val)*0.8)  
  val['id_item'] = [key for i in range(len(val))] 
  data_train = pd.concat([data_train, val[:a]], axis=0) 
  data_test = pd.concat([data_test,val[a:]], axis = 0) 

In [11]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# data user
def transform(data):
  tensor_id_user  = torch.Tensor(np.stack(data['idx_user'].values))
  tensor_id_item  = torch.Tensor(np.stack(data['idx_item'].values))
  tensor_rating  = torch.Tensor(np.stack(data['rating'].values))
  tensor_rec = torch.Tensor(np.stack(data['is_recommended'].values))

  dataset = TensorDataset(tensor_id_user, tensor_id_item, tensor_rating,tensor_rec)
  dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
  return dataloader

In [12]:
data_train.head()

,is_recommended,hours,idx_user,idx_item,rating,id_item
15124,1,195.0,0,13839,5.0,0
73260,1,253.0,0,16522,5.0,0
103924,1,172.0,0,46654,5.0,0
121375,1,69.0,0,16209,4.5,0
177489,1,63.0,0,1843,4.5,0


* transform data

In [13]:
dataloader_train , dataloader_test = transform(data_train), transform(data_test)

# training with implicit data

In [4]:
class MLP_layer(nn.Module):
    def __init__(self,embed_dim):
        super(MLP_layer,self).__init__()
        self.relu = nn.ReLU()
        self.dense1 = nn.Linear(in_features =embed_dim, out_features = 64)
        self.dense2 = nn.Linear(in_features = 64, out_features = 32)
        self.dense3 = nn.Linear(in_features = 32, out_features = 8)
        
    def forward(self, user_embed, item_embed):
        concat_features = torch.cat((user_embed,item_embed), dim = 1)
        x = self.dense1(concat_features)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        return x

class NeuralMF(nn.Module):
    def __init__(self,num_user, num_item, latent_mlp, latent_mf):
        super(NeuralMF,self).__init__()
        self.embedding_user_mlp = nn.Embedding(num_embeddings = num_user,embedding_dim = latent_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings = num_item,embedding_dim = latent_mlp)
    
        self.embedding_user_mf = nn.Embedding(num_embeddings = num_user, embedding_dim = latent_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings = num_item, embedding_dim = latent_mf)
        
        self.MLPlayer = MLP_layer(latent_mlp*2)
        self.dense = nn.Linear(in_features = 16, out_features = 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,user_idx,item_idx):
        user_embed_mlp = self.embedding_user_mlp(user_idx)
        item_embed_mlp = self.embedding_item_mlp(item_idx)
        
        user_embed_mf = self.embedding_user_mf(user_idx)
        item_embed_mf = self.embedding_item_mf(item_idx)
        element_wise = user_embed_mf*item_embed_mf
        MLP = self.MLPlayer(user_embed_mlp,item_embed_mlp)
        vector_cocat = torch.cat((element_wise,MLP),dim = 1)

        x = self.dense(vector_cocat)
        x = self.sigmoid(x)
        return x 

In [5]:
num_user = len(data_user)
num_item = len(data_item) 
latent_mlp = 10
latent_mf = 8

In [16]:
for batch in dataloader_train:
    print(batch)
    if True:
        break

[tensor([23230., 54971.,  8348.,  6711., 10630., 13479., 29456., 57277., 50085.,
        56992., 35414., 54487., 49140., 22185., 46829., 57530., 38247., 40991.,
        10194.,  3883., 27639., 54194., 17383., 37671.,  6447., 13136., 57195.,
        42390., 54558., 56721., 17435.,  6159., 28009., 15510., 18159., 49610.,
         8864., 34943., 29715., 53873., 27555., 20820., 40274., 56087., 18549.,
        44388., 42526., 52507., 43675., 34743.,  4867., 25059., 24610., 40544.,
        46309., 21534.,  6772., 18670., 34676., 42931., 52065., 12064., 53859.,
        17615., 52848.,   450., 21638., 11858., 57126., 13988., 46880.,  6149.,
        27110., 52685., 49786.,  8775., 42282., 31171.,  6747., 23223., 38443.,
         1821., 11773., 41789., 29336., 29586., 10156.,  3215., 35061., 41651.,
         4578., 33105., 20687.,  8671., 35475., 12483., 20255.,  4858., 11369.,
        56042., 35118., 21372., 28571., 26358., 53616., 22774., 30265.,  2722.,
        21147., 24889.,  5193., 41619.,

In [18]:
model = NeuralMF(num_user, num_item, latent_mlp, latent_mf)

In [19]:
print(model)

NeuralMF(
  (embedding_user_mlp): Embedding(57973, 10)
  (embedding_item_mlp): Embedding(49628, 10)
  (embedding_user_mf): Embedding(57973, 8)
  (embedding_item_mf): Embedding(49628, 8)
  (MLPlayer): MLP_layer(
    (relu): ReLU()
    (dense1): Linear(in_features=20, out_features=64, bias=True)
    (dense2): Linear(in_features=64, out_features=32, bias=True)
    (dense3): Linear(in_features=32, out_features=8, bias=True)
  )
  (dense): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
  (sigmoid): Sigmoid()
)


In [20]:
learning_rate = 0.002
epoch = 50
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr= learning_rate)

In [21]:
device = ('cuda' if torch.cuda.is_available()
            else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [26]:
# evaluation metric in total data
from sklearn.metrics import ndcg_score, f1_score

# evaluation metric
user = data_user['idx_user'].unique()
# ndcg, f1
def metric(model):
    a = np.random.randint(0,len(user)-500)
    user_eval = user[a:a+500]
    ndcg = 0
    f1 = 0 
    for u in user_eval:
        item = list(data[data['idx_user'] == u]['idx_item'])
        actual = list(data[data['idx_user'] == u]['is_recommended'])
        s = predict(model,u,item)

        ndcg += ndcg_score([actual],[s])
        f1 += f1_score(np.array(actual),(np.array(s) >= 0.9).astype(int))
    result_ndcg = ndcg/len(user_eval)
    result_f1 = f1/len(user_eval)
    return result_ndcg,result_f1
    
def predict(model,u_idx,item):
    
    score = []
    for i in item:
        predict = model(torch.Tensor([u_idx]).long(),torch.Tensor([i]).long())
        score.append(predict.item())
    return score

* loop train

In [27]:
for i in range(epoch):
    model.train()
    total_loss_train = 0
    for batch in dataloader_train:
        id_user,id_item,_,rec = batch
        id_user,id_item,rec = id_user.to(device), id_item.to(device), rec.to(device)

        output = model(id_user.long(),id_item.long())
        loss = loss_function(output.reshape(-1),rec.float())
        total_loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    total_loss_test = 0
    # evaluation
    ndcg,f1 = metric(model)
 
    print(f'Epoch{i +1}/{epoch}, loss_train : {total_loss_train/len(dataloader_train)}, metric_ndcg: {ndcg}, metric_f1:{f1}')

Epoch1/50, loss_train : 0.28172174323606125, metric_ndcg: 0.9772828043690475, metric_f1:0.7158609192801563
Epoch2/50, loss_train : 0.2712961802994211, metric_ndcg: 0.9810661180155545, metric_f1:0.733624821607788
Epoch3/50, loss_train : 0.26158550698829025, metric_ndcg: 0.9821165621138419, metric_f1:0.7601551949334521
Epoch4/50, loss_train : 0.2524581143324706, metric_ndcg: 0.976755053087086, metric_f1:0.7346033142334074
Epoch5/50, loss_train : 0.24400598435923332, metric_ndcg: 0.9815447241746403, metric_f1:0.7692880274815417
Epoch6/50, loss_train : 0.23605456829212862, metric_ndcg: 0.9805728628245324, metric_f1:0.800454321351614
Epoch7/50, loss_train : 0.22869432667704065, metric_ndcg: 0.982256408493869, metric_f1:0.7973937560351408
Epoch8/50, loss_train : 0.22181735237035688, metric_ndcg: 0.9805987570223623, metric_f1:0.7959538599422232
Epoch9/50, loss_train : 0.2155287012751823, metric_ndcg: 0.9813243136084027, metric_f1:0.8183383437119708
Epoch10/50, loss_train : 0.20957054254958352

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch12/50, loss_train : 0.19912319106177342, metric_ndcg: 0.9802034704830086, metric_f1:0.8188031061188297
Epoch13/50, loss_train : 0.19452021914412673, metric_ndcg: 0.9827594039498458, metric_f1:0.8323626299648299


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch14/50, loss_train : 0.19007100469132795, metric_ndcg: 0.9788478826197583, metric_f1:0.8549448738214221
Epoch15/50, loss_train : 0.18596994924946514, metric_ndcg: 0.9825222872682756, metric_f1:0.8632169914748283
Epoch16/50, loss_train : 0.1821484308144353, metric_ndcg: 0.9798234711933961, metric_f1:0.8445282077466123
Epoch17/50, loss_train : 0.17855054625741457, metric_ndcg: 0.9782886013280349, metric_f1:0.8483194250869737
Epoch18/50, loss_train : 0.1752165375085171, metric_ndcg: 0.9799740343223554, metric_f1:0.859103337187996
Epoch19/50, loss_train : 0.1721914964801459, metric_ndcg: 0.9815153682984968, metric_f1:0.86883898347539
Epoch20/50, loss_train : 0.16925187578218548, metric_ndcg: 0.9817720699776614, metric_f1:0.8716065314564828
Epoch21/50, loss_train : 0.16650270287893307, metric_ndcg: 0.9827641732383081, metric_f1:0.8718159797133881
Epoch22/50, loss_train : 0.16389539197591035, metric_ndcg: 0.9776113382968273, metric_f1:0.848461565072048
Epoch23/50, loss_train : 0.16137376

...

Epoch29/50, loss_train : 0.14862697224912988, metric_ndcg: 0.9783797331440467, metric_f1:0.8760716793673882


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch30/50, loss_train : 0.14698128574112782, metric_ndcg: 0.9765673801862044, metric_f1:0.8877304045083858
Epoch31/50, loss_train : 0.14538948221572198, metric_ndcg: 0.9809370940198654, metric_f1:0.8928911048758261
Epoch32/50, loss_train : 0.14380462866679428, metric_ndcg: 0.9824572195492524, metric_f1:0.8964170774980135
Epoch33/50, loss_train : 0.14221430691983025, metric_ndcg: 0.9795678032074732, metric_f1:0.8903816068837117
Epoch34/50, loss_train : 0.1406639507449399, metric_ndcg: 0.9804670238621126, metric_f1:0.8965856560896465
Epoch35/50, loss_train : 0.13918448041759257, metric_ndcg: 0.9808528546238817, metric_f1:0.891212426374695
Epoch36/50, loss_train : 0.13801014411404525, metric_ndcg: 0.978934563018909, metric_f1:0.886136173539207
Epoch37/50, loss_train : 0.1365915853018666, metric_ndcg: 0.981270480494297, metric_f1:0.8927759905076538
Epoch38/50, loss_train : 0.1352050009166874, metric_ndcg: 0.980215922119358, metric_f1:0.8885157999749084
Epoch39/50, loss_train : 0.134135991

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Epoch44/50, loss_train : 0.12851299438191627, metric_ndcg: 0.9775130352728778, metric_f1:0.9024554237648179
Epoch45/50, loss_train : 0.127583197179801, metric_ndcg: 0.9799959424891859, metric_f1:0.8923825338886615
Epoch46/50, loss_train : 0.12662925723398347, metric_ndcg: 0.9795778815470716, metric_f1:0.8966533890480686
Epoch47/50, loss_train : 0.12563357158241156, metric_ndcg: 0.9788161407012694, metric_f1:0.8997055129158141
Epoch48/50, loss_train : 0.12479725217459058, metric_ndcg: 0.9813286458995665, metric_f1:0.9067247018885041
Epoch49/50, loss_train : 0.1238996316040367, metric_ndcg: 0.9799114364660246, metric_f1:0.9028410887146398
Epoch50/50, loss_train : 0.12302231631897552, metric_ndcg: 0.9788268384124682, metric_f1:0.8979786584085819


* save model

In [ ]:
torch.save(model.state_dict(),'model_NeuMF_implicit_1.pth')

# evaluation metric ndcg , f1 in test

In [17]:
model_2 = NeuralMF(num_user, num_item, latent_mlp, latent_mf)
model_2.load_state_dict(torch.load('/kaggle/input/data-path-model/model_NeuMF_implicit.pth'))

<All keys matched successfully>

In [47]:
from sklearn.metrics import ndcg_score, f1_score
def metric_2(model,k = 10):
    user = list(data_test['idx_user'].unique())
    a = np.random.randint(0,len(user)-500)
    user_eval = user[a:a+500]
    ndcg = 0
    f1 = 0
    for u in user_eval:
        u_interac_item = list(data_test[data_test['idx_user'] == u]['idx_item'])
        score = predict_2(model,u,u_interac_item)
        score_item = {u_interac_item[i]:score[i] for i in range(len(score))}
        score_item = dict(sorted(score_item.items(), key=lambda item: item[1]))
        new_item = list(score_item.keys())[:k]
        new_score = list(score_item.values())[:k]
        actual = list(data_test[(data_test['idx_item'].isin(new_item)) & (data_test['idx_user'] == u)]['is_recommended'])
        ndcg += ndcg_score([actual],[new_score])
        f1 += f1_score(np.array(actual),np.array([1 if new_score[i]>= 0.75 else 0 for i in range(len(new_score))]))
    result_ndcg = ndcg/len(user_eval)
    result_f1 = f1/len(user_eval)
    return result_ndcg,result_f1
    
def predict_2(model,u_idx,item):
    model.eval()
    score = []
    for i in item:
        predict = model(torch.Tensor([u_idx]).long(),torch.Tensor([i]).long())
        score.append(predict.item())
    return score

In [51]:
# evaluation in random  500 user
for i in range(5):
    ndcg,f1 = metric_2(model_2,k=10)  
    
    print('evaluation '+ str(i+1) + ' ndcg@10: '+ str(ndcg) + ' f1@10: ' + str(f1))

evaluation 1 ndcg@10: 0.908144382424194 f1@10: 0.5548140804156291


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


evaluation 2 ndcg@10: 0.9094878059872824 f1@10: 0.5782862412785018


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


evaluation 3 ndcg@10: 0.9173281452737037 f1@10: 0.5695373966075213
evaluation 4 ndcg@10: 0.9123053428200563 f1@10: 0.5758121490996106


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


evaluation 5 ndcg@10: 0.9219051446829736 f1@10: 0.5852188760843566
